<a href="https://colab.research.google.com/github/vishaljoshi24/Dungeons-and-Dragons-Turn-Classification/blob/main/classifying_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U dspy

SET-UP

In [ ]:
import dspy
from dspy.clients.lm_local import LocalProvider

lm = dspy.LM("huggingface/meta-llama/Llama-3.2-1B", provider=LocalProvider(), max_tokens=2000)
dspy.configure(lm=lm)

DATA

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("garrykuwanto/crd3_training_pairs", download_mode="force_redownload")

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
query = []
passage = []
for i in range(len(df.train[0:])):
  query.append(df.train[i]['query'])
  passage.append(df.train[i]['passage'])

df1 = pd.DataFrame({'context': query, 'current turn': passage})

In [ ]:
df1

In [ ]:
df1 = df1[(~df1['current turn'].str.startswith('MATT:')) & (~df1['current turn'].str.startswith('DM Responds:')) & (~df1['current turn'].str.startswith('MATT Responds:')) ]

In [ ]:
df1

In [ ]:
random_data = df1.sample(n=300)
df2 = random_data

df2

In [ ]:
df2.to_json('unlabelled_trainset,json', orient='split', compression='infer')

In [ ]:

examples = []

for context, turn, quality in df1.values:
  examples.append(dspy.Example(context=context, turn=turn, quality=quality).with_inputs("context", "current turn"))

examples

SIGNATURES

In [ ]:
from typing import Literal


from typing import Literal

class TurnClassifier(dspy.Signature):
    data: dict[str, str] = dspy.InputField(desc = """
  {'context': 'The three previous game turns which describe a player's action or their dialogue.',
   'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
   }""")
    behaviour_class: Literal['collaborative',
                             'contextually-relevant',
                             'goal-oriented',
                             'open-ended'] = dspy.OutputField(desc="The feature of a player's turn within a Dungeons & Dragons game: collaborative, open-ended, goal-oriented, contextually relevant")

classify = dspy.ChainOfThought(BehaviourClassifier)

